**Goal: locate a submarine in the Puget Sound using noisy acoustic data**

Sarayu Gundlapalli
<br>
saru07g@w.edu

**Tasks:**

1. Through averaging of the Fourier transform and visual inspection, determine the frequency signature (center frequency) generated by the submarine.

In [86]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
from numpy.fft import fftn, ifftn

# Load data
data = np.load('subdata.npy')

L = 10;
N_grid = 64; 
xx = np.linspace(-L, L, N_grid) #spatial grid in x dir
x = xx[0:N_grid]
y = x # same grid in y,z direction
z = x

K_grid = (2*np.pi/(2*L))*np.linspace(-N_grid/2, N_grid/2 -1, N_grid) # draft 3

#3D Meshgrid
Kx, Ky, Kz = np.meshgrid(K_grid, K_grid, K_grid, indexing='ij')

avg_fft = np.zeros((N_grid, N_grid, N_grid), dtype=complex)

for i in range(0,49):
    signal = np.reshape(data[:, i], (N_grid, N_grid, N_grid))
    fft_data = np.fft.fftn(signal)
    
    avg_fft += fft_data

# Calculate the average FFT result
avg_fft = avg_fft / 49
avg_fft = np.abs(avg_fft)
avg_fft = np.fft.fftshift(avg_fft)

[max_x, max_y, max_z] = np.unravel_index(np.argmax(avg_fft), avg_fft.shape)
#print(max_loc)

#find center frequencies
center_freq = [K_grid[max_x], K_grid[max_y], K_grid[max_z]]

#diction = {"Center Frequency:":center_freq, "Index of Max:": [max_x,max_y,max_z]}
#pd.DataFrame.from_dict(diction)

2. Design and implement a Filter to extract this frequency signature in order to denoise the data and determine the path of the submarine.
3. Determine and plot the 𝑥, 𝑦 coordinates of the submarine during the 24 hour period. This information can be used to deploy a sub-tracking aircraft to keep an eye on your submarine in the future.

Note: All plotting code has been commented out.

In [84]:
def plot_data(data, sigma):
    N_grid = 64
    L = 10
    sgrid = np.linspace(-L, L, N_grid) #spatial grid

    K_grid = (2*np.pi/(2*L))*np.linspace(-N_grid/2, N_grid/2 -1, N_grid) # K_grid
    Kx, Ky, Kz = np.meshgrid(K_grid, K_grid, K_grid, indexing='ij')

    #def gaussian function using center frequencies
    def g(kx, ky, kz, center, sigma):
        tau = 1 / (2 * (sigma ** 2))
        gaussian = np.exp(-tau * ((kx - center[0]) ** 2 + (ky - center[1]) ** 2 + (kz - center[2]) ** 2))
        return gaussian

    GF = g(Kx, Ky, Kz, center_freq, sigma)
    
    filtered = np.zeros((49,3))
    unfiltered = np.zeros((49,3))
    
    for j in range(49):
        sig = data[:, j].reshape((N_grid, N_grid, N_grid)) 
        freq = np.fft.fftshift(fftn(sig))
        filtered_freq = GF * freq #multiply elementwise with gaussian
        filtered_sig = np.abs(ifftn(filtered_freq))
        
        [xf, yf, zf] = np.unravel_index(np.argmax(filtered_sig), filtered_sig.shape)                       
        filtered[j] = [sgrid[xf], sgrid[yf], sgrid[zf]]
        
        max_val_unfiltered = np.abs(sig).flatten().argmax()
        
        xu, yu, zu = np.unravel_index(max_val_unfiltered, sig.shape)
        unfiltered[j] = [sgrid[xu], sgrid[yu], sgrid[zu]]
    #print(xf, yf, zf)
    #print(sgrid[xf], sgrid[yf], sgrid[zf])


 #plotting code
    #plt.plot(unfiltered[:, 1], unfiltered[:, 0], alpha=0.5, linestyle='-', marker='.')
    #plt.plot(filtered[:, 1], filtered[:, 0], linestyle='--', marker='.')
    #plt.scatter(filtered[0, 1], filtered[0, 0], c='red', marker='s', label='end')
    #plt.scatter(filtered[-1, 1], filtered[-1, 0], c='green', marker='^', label='start')
    #plt.xlabel('X Position')
    #plt.ylabel("Y Position")
    #plt.title(f'Submarine X-Y Positions (Filtered and Unfiltered with sigma={sigma})')
    #plt.legend(["Unfiltered Data", "Filtered Data", "end", "start"], loc="upper right")
    #plt.grid(True)

    #plt.tight_layout()
    #plt.show()

    #plt.tight_layout()
    #plt.show()

In [85]:
data = np.load("subdata.npy")
#plot data for set of sigma values
sigma_values = [.5, 2, 9, 100]
for sigma in sigma_values:
    plot_data(subdata, sigma=sigma)